In [102]:
#Plotting 
def plot_snd_throughputs(owd, container, duration, range_, flownum, labels, legends, pdf_num, veth_str) :
    
    range_str       = str(range_)
    duration_str    = str(duration)
    flownum_str     = str(flownum)
    pdf_num_str     = str(pdf_num)
    
    throughput_file = "../saves/" + container + "/csv/" + veth_str + "_owd"  + str(owd) + "_" + pdf_num_str + ".csv"
    plot_file       = "../gnuplots/snd_throughputs.plot"
    output          = "../saves/" + container + "/pdfs/" + veth_str + "_owd" + str(owd) + "_" + pdf_num_str + ".pdf"

    if(labels and legends) :
        !gnuplot -e "labels='1'" -e "legends='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file
    elif(labels) : 
        !gnuplot -e "labels='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file
    elif(legends) : 
        !gnuplot -e "legends='1'" -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file
    else : 
        !gnuplot -e "output_file='$output'" -e "throughput_file='$throughput_file'" -e "range='$range_str'" -e "flownum='$flownum_str'" -e "duration='$duration_str'" $plot_file


In [107]:
from shutil import copyfile
import os.path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

#range_ = 1200
#owd = 100
#container = "rmcat1"
def eval_measurements(owd, container, time_range_):
    save_directory    = "../saves/"+container+"/csv/"
    owd_str           = "owd" + str(owd)
    columns = ['sender_rate','fec_rate','capacity']
    alldf = pd.DataFrame(columns=columns)
    alldf = alldf.fillna(0) # with 0s rather than NaNs
    for x in range(0, 9):
        csv_file = save_directory + "veth_stats_" + owd_str + "_" + str(x) + ".csv"
        if(os.path.isfile(csv_file) == False) :
            print('Missing file: {}'.format(csv_file))
            continue

        df = pd.read_csv(csv_file, names=columns)
        alldf = pd.concat([alldf, df[:time_range_]])
    
    print("\nAverages\n" + str(alldf.mean()))
    print("\nStandard deviations\n" + str(alldf.std()))



In [95]:
from shutil import copyfile
import os.path

def save_measurement(owd, container, counter) :
    
    save_directory    = "../saves/"+container+"/csv/"
    owd_str           = "owd" + str(owd)
    veths             = ["veth0_stats","veth3_stats"]
    
    for veth in veths : 
        src_file = veth + ".csv"
        dst_file = veth + "_" + owd_str + "_" + str(counter) + ".csv"
        src_path = "../temp/" + src_file
        dst_path = save_directory + dst_file
        copyfile(src_path, dst_path)


In [135]:
#RMCAT1 Test

container = "rmcat1"
veth0_str = "veth0_stats"
duration  = 120
range_    = 3000
flownum   = 1

range_bgn = 3
range_end = 4
plays     = [0,1,0] 

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(range_bgn, range_end):
        !cd ..;cd ..; scripts2/runs/rmcat1.sh -s $owd 
        save_measurement(owd, container, y)
        plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
    
    eval_measurements(owd, container, duration * 10)
    
    

[INFO] Constructing NTRT system machine.
[INFO] NTRT system machine is constructed
[INFO] Constructing Managed datamap.
[INFO] Managed datamap is constructed
[INFO] FSM: Setup event is fired, actual state is Halt
[INFO] Constructing PCAP Sniffer component.
[INFO] PCAP Sniffer component is constructed
[INFO] Constructing Sniff Evaluator component.
[INFO] Sniff Evaluator component is constructed
[INFO] Constructing Records Recycle component.
[INFO] Records Recycle component is constructed
[INFO] Constructing Recorder component.
[INFO] Recorder component is constructed
[INFO] Constructing Records relayer component.
[INFO] Records relayer component is constructed
[INFO] Constructing Record Accumulator component.
[INFO] Record Accumulator component is constructed
[INFO] Constructing Groupcounter component.
[INFO] Groupcounter component is constructed
[INFO] Constructing Command File Executor component.
[INFO] Command File Executor component is constructed
[INFO] Feature IP_PACKETS          

In [111]:
#RMCAT2 Test

container = "rmcat2"
veth0_str = "veth0_stats"
duration  = 140
range_    = 5000
flownum   = 1

range_bgn = 3
range_end = 4
plays     = [0,1,0] 

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(range_bgn, range_end):
        !cd ..;cd ..; scripts2/runs/rmcat2.sh -s $owd 
        save_measurement(owd, container, y)
        plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
    
    eval_measurements(owd, container, duration * 10)

[INFO] Constructing NTRT system machine.
[INFO] NTRT system machine is constructed
[INFO] Constructing Managed datamap.
[INFO] Managed datamap is constructed
[INFO] FSM: Setup event is fired, actual state is Halt
[INFO] Constructing PCAP Sniffer component.
[INFO] PCAP Sniffer component is constructed
[INFO] Constructing Sniff Evaluator component.
[INFO] Sniff Evaluator component is constructed
[INFO] Constructing Records Recycle component.
[INFO] Records Recycle component is constructed
[INFO] Constructing Recorder component.
[INFO] Recorder component is constructed
[INFO] Constructing Records relayer component.
[INFO] Records relayer component is constructed
[INFO] Constructing Record Accumulator component.
[INFO] Record Accumulator component is constructed
[INFO] Constructing Command File Executor component.
[INFO] Command File Executor component is constructed
Joining session 0x1e8e7d0
Connecting to 5000/5001/5005/5011/5012/5010
New pad: recv_rtp_sink_0, looking for recv_rtp_src_0_*

In [104]:
#RMCAT3 Test

container = "rmcat3"
veth0_str = "veth0_stats"
veth3_str = "veth3_stats"
duration  = 100
range_    = 5000
flownum   = 1

range_bgn = 3
range_end = 4
plays     = [0,1,0] 

owds      = [50,150,300]
labels    = [True,False,False]
legends   = [False,False,True]

for x in range(0,3) :
    
    if(plays[x] == 0) :
        continue;
        
    owd    = owds[x]
    label  = labels[x]
    legend = legends[x]
    
    for y in range(range_bgn, range_end):
        !cd ..;cd ..; scripts2/runs/rmcat2.sh -s $owd 
        save_measurement(owd, container, y)
        plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth0_str)
        plot_snd_throughputs(owd, container, duration, range_, flownum, label, legend, y, veth3_str)
    
    eval_measurements(owd, container, duration * 10)

[INFO] Constructing NTRT system machine.
[INFO] NTRT system machine is constructed
[INFO] Constructing Managed datamap.
[INFO] Managed datamap is constructed
[INFO] FSM: Setup event is fired, actual state is Halt
[INFO] Constructing PCAP Sniffer component.
[INFO] PCAP Sniffer component is constructed
[INFO] Constructing Sniff Evaluator component.
[INFO] Sniff Evaluator component is constructed
[INFO] Constructing Records Recycle component.
[INFO] Records Recycle component is constructed
[INFO] Constructing Recorder component.
[INFO] Recorder component is constructed
[INFO] Constructing Records relayer component.
[INFO] Records relayer component is constructed
[INFO] Constructing Record Accumulator component.
[INFO] Record Accumulator component is constructed
[INFO] Constructing Command File Executor component.
[INFO] Command File Executor component is constructed
[INFO] Feature IP_PACKETS                          is installed
[INFO] Feature IP_BYTES                            is instal